In [1]:
import numpy as np
import pickle
import cv2
import os
import imutils

detector = cv2.dnn.readNetFromCaffe("deploy.prototxt.txt","res10_300x300_ssd_iter_140000.caffemodel")
embedder = cv2.dnn.readNetFromTorch("openface_nn4.small2.v1.t7")
recognizer = pickle.loads(open("recognizer.pickle", "rb").read())
le = pickle.loads(open("le.pickle", "rb").read())

 
capture=cv2.VideoCapture(0)
present=[]
dic={}
while(True):
    _,frame=capture.read()
    frame = imutils.resize(frame, width=600)
    (h, w) = frame.shape[:2]

    imageBlob = cv2.dnn.blobFromImage(cv2.resize(frame, (300, 300)), 1.0, (300, 300),(104.0, 177.0, 123.0), swapRB=False, crop=False)

    detector.setInput(imageBlob)
    detected_faces = detector.forward()
    
    for i in range(0, detected_faces.shape[2]):
        confidence = detected_faces[0, 0, i, 2]
        if confidence > 0.6:
            box = detected_faces[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")
            face = frame[startY:endY, startX:endX]
            (fH, fW) = face.shape[:2]
            
            if fW < 20 or fH < 20:
                continue

            faceBlob = cv2.dnn.blobFromImage(face, 1.0 / 255,(96, 96), (0, 0, 0), swapRB=True, crop=False)
            embedder.setInput(faceBlob)
            vec = embedder.forward()

            preds = recognizer.predict_proba(vec)[0]
            j = np.argmax(preds)
            proba = preds[j]
            name = le.classes_[j]
            
            if name in dic:
                dic[name]+=1
            else:
                dic[name]=1

            text = "{}: {:.2f}%".format(name, proba * 100)
            y = startY - 10 if startY - 10 > 10 else startY + 10
            cv2.rectangle(frame, (startX, startY), (endX, endY),(0, 0, 255), 2)
            cv2.putText(frame, text, (startX, y),cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)

        
    cv2.imshow('Recognizing faces',frame)
    if cv2.waitKey(1)==13:
        break

for roll_no in dic:
    if(dic[roll_no]>=100):
        present.append(roll_no)
        
if "unknown" in present: 
    present.remove("unknown")
print("present student are :")
print(present)       
capture.release()
cv2.destroyAllWindows()
capture.release()
cv2.destroyAllWindows()

present student are :
['madhuri']
